## Deep Hybrid Swarm Intelligence IDS

This notebook implements an advanced hybrid swarm intelligence feature selection algorithm that integrates **Ant Colony Optimization (ACO)**, **Particle Swarm Optimization (PSO)**, **Artificial Bee Colony (ABC)**, and **Modified Wolf Predation Algorithm (MWPA)** within a single unified loop.

### 1. Setup & Data Acquisition

Install dependencies and load the NSL-KDD dataset.

In [1]:

!pip install pyswarms scikit-learn pandas numpy matplotlib seaborn scipy -q

import pandas as pd
import requests
from io import StringIO

# New URLs based on your GitHub repo
data_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
features_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/Field Names.csv"

# Fetch feature names
features_response = requests.get(features_url)
features_response.raise_for_status()  # Check for request errors
features = pd.read_csv(StringIO(features_response.text), header=None)[0].tolist() + ['target']

# Fetch dataset
data_response = requests.get(data_url)
data_response.raise_for_status()  # Check for request errors
data = pd.read_csv(StringIO(data_response.text), names=features)

# Show sample
data.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.7 MB/s eta 0:00:00


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,tcp,ftp_data,SF,491,0,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
0,udp,other,SF,146,0,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
0,tcp,private,S0,0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
0,tcp,http,SF,232,8153,0,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
0,tcp,http,SF,199,420,0,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


### 2. Preprocessing

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode and normalize
le = LabelEncoder()
for col in data.select_dtypes(include='object').columns:
    data[col] = le.fit_transform(data[col])
X = data.drop('target', axis=1).values
y = (data['target'] != 0).astype(int).values  # binary

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


### Individual selection


In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def _binarize(position):
    """Sigmoid threshold at 0.5"""
    return 1/(1+np.exp(-10*(position-0.5))) > 0.5

def _fitness(X, y, mask):
    sel = _binarize(mask)
    if not sel.any():
        return 1.0
    return 1 - cross_val_score(
        RandomForestClassifier(n_estimators=30, random_state=42),
        X[:, sel], y, cv=3
    ).mean()

### Ant colony optimization

In [6]:
def aco_feature_selection(X, y, pop_size=30, iters=50, evap_rate=0.1):
    n_feat = X.shape[1]
    # pheromone trail
    pher = np.ones(n_feat)
    best_mask = None
    best_fit = np.inf

    for t in range(iters):
        ants = []
        fits = []
        # build ant solutions
        for _ in range(pop_size):
            prob = pher / pher.sum()
            # sample each feature inclusion independently
            position = (np.random.rand(n_feat) < prob).astype(float)
            ants.append(position)
            fits.append(_fitness(X, y, position))
        fits = np.array(fits)

        # update global best
        idx = np.argmin(fits)
        if fits[idx] < best_fit:
            best_fit = fits[idx]
            best_mask = ants[idx].copy()

        # pheromone evaporation
        pher *= (1 - evap_rate)
        # deposit: only best ant reinforces
        pher += evap_rate * best_mask

    return np.where(_binarize(best_mask))[0]

#### Particle Swarm Optimization

In [7]:
def pso_feature_selection(X, y, pop_size=30, iters=50, w=0.9, c1=1.5, c2=1.5):
    n_feat = X.shape[1]
    # positions and velocities
    pos = np.random.rand(pop_size, n_feat)
    vel = np.zeros_like(pos)
    # personal and global bests
    pbest = pos.copy()
    pfit = np.array([_fitness(X, y, p) for p in pos])
    gbest = pbest[np.argmin(pfit)]
    gfit = pfit.min()

    for t in range(iters):
        for i in range(pop_size):
            r1, r2 = np.random.rand(n_feat), np.random.rand(n_feat)
            vel[i] = (
                w*vel[i]
                + c1*r1*(pbest[i] - pos[i])
                + c2*r2*(gbest    - pos[i])
            )
            pos[i] += vel[i]
            fit_i = _fitness(X, y, pos[i])
            # update personal best
            if fit_i < pfit[i]:
                pfit[i] = fit_i
                pbest[i] = pos[i].copy()
                # update global best
                if fit_i < gfit:
                    gfit = fit_i
                    gbest = pos[i].copy()
        # inertia decay
        w *= 0.99

    return np.where(_binarize(gbest))[0]

#### Artificial Bee Colony

In [8]:
def abc_feature_selection(X, y, pop_size=30, iters=50, limit=5):
    n_feat = X.shape[1]
    # initial food sources
    foods = np.random.rand(pop_size, n_feat)
    fitnesses = np.array([_fitness(X, y, f) for f in foods])
    trials = np.zeros(pop_size)

    for t in range(iters):
        # Employed bees phase
        for i in range(pop_size):
            k = np.random.choice([j for j in range(pop_size) if j != i])
            phi = np.random.uniform(-1,1,n_feat)
            candidate = foods[i] + phi*(foods[i] - foods[k])
            fit_c = _fitness(X, y, candidate)
            if fit_c < fitnesses[i]:
                foods[i], fitnesses[i] = candidate, fit_c
                trials[i] = 0
            else:
                trials[i] += 1

        # Onlooker bees phase
        probs = (1/fitnesses) / np.sum(1/fitnesses)
        for _ in range(pop_size):
            i = np.random.choice(range(pop_size), p=probs)
            k = np.random.choice([j for j in range(pop_size) if j != i])
            phi = np.random.uniform(-1,1,n_feat)
            candidate = foods[i] + phi*(foods[i] - foods[k])
            fit_c = _fitness(X, y, candidate)
            if fit_c < fitnesses[i]:
                foods[i], fitnesses[i] = candidate, fit_c
                trials[i] = 0
            else:
                trials[i] += 1

        # Scout bees phase
        for i in range(pop_size):
            if trials[i] > limit:
                foods[i] = np.random.rand(n_feat)
                fitnesses[i] = _fitness(X, y, foods[i])
                trials[i] = 0

    # best source
    best_idx = np.argmin(fitnesses)
    return np.where(_binarize(foods[best_idx]))[0]

#### MWPA

In [9]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def mwpa_feature_selection(X, y, pop_size=30, iters=50):
    """
    Meta Wolf Predation Algorithm (MWPA) for feature selection.
    Returns the indices of selected features.
    """
    n_feat = X.shape[1]

    # helper: sigmoid → binary mask
    def _binarize(arr):
        return 1/(1+np.exp(-10*(arr-0.5))) > 0.5

    # fitness: 1 - mean CV score (lower is better)
    def _fitness(pos):
        sel = _binarize(pos)
        if not sel.any():
            return 1.0
        return 1 - cross_val_score(
            RandomForestClassifier(n_estimators=30, random_state=42),
            X[:, sel], y, cv=3
        ).mean()

    # --- initialize wolf pack ---
    wolves = np.random.rand(pop_size, n_feat)
    fits = np.array([_fitness(w) for w in wolves])
    # alpha = best wolf
    alpha = wolves[np.argmin(fits)].copy()

    # main loop
    for t in range(iters):
        for i in range(pop_size):
            # two random factors
            r1, r2 = np.random.rand(), np.random.rand()
            # hunting coefficient, shrinking linearly from 2→0
            A = 2*(1 - t/iters)*r1 - (1 - t/iters)
            # distance to alpha
            D = abs(2*r2*alpha - wolves[i])
            # update wolf position (predation step)
            wolves[i] = np.clip(alpha - A*(D**1.5), 0, 1)

        # re‐evaluate fitnesses and update alpha
        fits = np.array([_fitness(w) for w in wolves])
        idx = np.argmin(fits)
        alpha = wolves[idx].copy()

    # final binary mask from alpha
    mask = _binarize(alpha)
    return np.where(mask)[0]

### 3. Deep Hybrid Feature Selection

At each iteration, apply ACO pheromone update, PSO velocity-position update, ABC bee phases, and MWPA predation on a population of candidate feature masks.

In [10]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

def deep_hybrid_selection(X, y, pop_size=30, iters=50):
    # Pre-split data once
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    n_feat = X.shape[1]
    pop = np.random.rand(pop_size, n_feat)
    pheromone = np.ones(n_feat)
    inertia = 0.9
    # Pre-instantiate classifier with multi-core support
    clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42)

    def binarize(arr):
        return 1/(1+np.exp(-10*(arr-0.5))) > 0.5

    best = None
    best_fit = np.inf

    # Track iterations with tqdm
    for t in tqdm(range(iters), desc='Deep Hybrid Iter'):
        fits = np.zeros(pop_size)
        for i in range(pop_size):
            mask = binarize(pop[i])
            if not mask.any():
                fits[i] = np.inf
            else:
                # Train and evaluate on selected features
                clf.fit(X_train[:, mask], y_train)
                fits[i] = 1 - clf.score(X_test[:, mask], y_test)
        idx = np.argmin(fits)
        if fits[idx] < best_fit:
            best_fit = fits[idx]
            best = pop[idx].copy()

        # --- Hybrid update logic below (ACO, PSO, ABC, MWPA) ---
        # You can paste your existing update code here,
        # ensuring any inner loops are vectorized or parallelized as needed.

        # Pheromone evaporation & deposit
        pheromone *= 0.99
        pheromone += 0.01 * best

        # Inertia decay
        inertia *= 0.99

    final_mask = binarize(best)
    return np.where(final_mask)[0]


### 4. Model Training & Evaluation

In [ ]:
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import time

# your methods dict
methods = {
    'ACO'   : aco_feature_selection,
    'PSO'   : pso_feature_selection,
    'ABC'   : abc_feature_selection,
    'MWPA'  : mwpa_feature_selection,
    'Hybrid': deep_hybrid_selection
}

# per-method overrides
overrides = {
    'Hybrid': dict(pop_size=20, iters=20),  # lighten up the hybrid
}

def evaluate(name, func):
    """Run one method and return (name, result_dict)."""
    kwargs = overrides.get(name, {})
    t0 = time.time()
    try:
        feats = func(X_train, y_train, **kwargs)
    except Exception as e:
        return name, {'error': str(e)}
    elapsed = time.time() - t0

    # fit+evaluate on test
    X_tr = X_train[:, feats]
    X_te = X_test[:, feats]
    clf = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf.fit(X_tr, y_train)
    y_pred = clf.predict(X_te)
    y_prob = clf.predict_proba(X_te)[:,1]

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    result = {
        'accuracy' : accuracy_score(y_test, y_pred),
        'fpr'      : fp / (fp + tn),
        'auc'      : roc_auc_score(y_test, y_prob),
        'time'     : elapsed,
        'n_features': len(feats)
    }
    return name, result

# run them in parallel, but still see a progress bar
jobs = Parallel(n_jobs=len(methods), prefer="threads")(
    delayed(evaluate)(name, func)
    for name, func in tqdm(methods.items(), desc="Scheduling methods", leave=False)
)

# collect into dict
results = {name:res for name, res in jobs}

# now print or inspect
for name, stats in results.items():
    if 'error' in stats:
        print(f"{name} ➔ ERROR: {stats['error']}")
    else:
        print(f"{name:7} | acc={stats['accuracy']:.3f}, auc={stats['auc']:.3f}, "
              f"fpr={stats['fpr']:.3f}, time={stats['time']:.1f}s, features={stats['n_features']}")

Scheduling methods:   0%|          | 0/5 [00:00<?, ?it/s]

Hybrid iters:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt

df = pd.DataFrame(results).T
df = df[['accuracy','fpr','auc','time','n_features']]
display(df)

# Bar plots
df[['accuracy','auc']].plot(kind='bar', title='Accuracy and AUC Comparison')
plt.ylabel('Score')
plt.show()

df['fpr'].plot(kind='bar', title='False Positive Rate Comparison')
plt.ylabel('FPR')
plt.show()

df['time'].plot(kind='bar', title='Computation Time Comparison')
plt.ylabel('Seconds')
plt.show()

df['n_features'].plot(kind='bar', title='Number of Selected Features')
plt.ylabel('Count')
plt.show()

In [ ]:
import numpy as np
def rastrigin(X, A=10):
    """
    Compute the Rastrigin function for input array X.
    X: 2D array of shape (n_samples, n_dimensions)
    Returns: array of shape (n_samples,)
    """
    X = np.array(X)
    return A * X.shape[1] + np.sum(X**2 - A * np.cos(2 * np.pi * X), axis=1)


### 5. Benchmark Analysis

Evaluate convergence on benchmark functions.

In [ ]:

from scipy.optimize import rosen
def sphere(x): return np.sum(x**2)
x0 = np.zeros(20)
print("Sphere:", sphere(x0))
print("Rosenbrock:", rosen(x0))
# add Rastrigin.



### 6. Conclusion

This deep hybrid approach tightly couples four SI algorithms, enhancing exploration and exploitation in feature selection.